In [ ]:

import numpy as np
import pandas as pd 


In [ ]:
!pip install seaborn

In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf


In [ ]:
sample_submission=pd.read_csv(r"C:\Users\suhas\OneDrive\Documents\projects\prodigy\first prj\house-prices-advanced-regression-techniques\sample_submission.csv")
train=pd.read_csv(r"C:\Users\suhas\OneDrive\Documents\projects\prodigy\first prj\house-prices-advanced-regression-techniques\train.csv")
test=pd.read_csv(r"C:\Users\suhas\OneDrive\Documents\projects\prodigy\first prj\house-prices-advanced-regression-techniques\test.csv")

In [ ]:
train.columns

In [ ]:
train['Electrical']

In [ ]:
y=train['SalePrice']
X=train.drop('SalePrice',axis=1)

In [ ]:
len(train.columns),len(X.columns)

In [ ]:
X.columns

In [ ]:
y

In [ ]:
X=X.dropna(axis=1)
test=test.dropna(axis=1)

In [ ]:
len(X.columns),len(test.columns)

In [ ]:
list(set(X.columns)-set(test.columns))

In [ ]:
X=X.drop(['TotalBsmtSF',
 'KitchenQual',
 'GarageCars',
 'Utilities',
 'Functional',
 'Exterior2nd',
 'Exterior1st',
 'BsmtFinSF1',
 'BsmtHalfBath',
 'BsmtFinSF2',
 'BsmtUnfSF',
 'SaleType',
 'BsmtFullBath',
 'GarageArea',
 'MSZoning'],axis=1)

In [ ]:
len(X.columns),len(test.columns)

In [ ]:
set(test.columns)-set(X.columns)

In [ ]:
test=test.drop(['Electrical'],axis=1)

In [ ]:
len(test.columns),len(X.columns)

In [ ]:
categorical_columns=[col for col in X.columns if X[col].dtype=='object']
numerical_columns=[col for col in X.columns if X[col].dtype!='object']

In [ ]:
categorical_columns

In [ ]:
numerical_columns

In [ ]:
len(test.columns),len(X.columns)

In [ ]:
set(test.columns)-set(X.columns)

In [ ]:
X['Street'].value_counts(),test['Street'].value_counts()

In [ ]:
from sklearn.preprocessing import OrdinalEncoder

In [ ]:
enc=OrdinalEncoder()
ord_encoded=enc.fit_transform(X[categorical_columns])
ord_tested=enc.fit_transform(test[categorical_columns])

In [ ]:
ord_encoded.shape,ord_tested.shape

In [ ]:
X[categorical_columns].columns

In [ ]:
test[categorical_columns].columns

In [ ]:
X[categorical_columns].columns

In [ ]:
enc.get_feature_names_out(categorical_columns).shape

In [ ]:
ord_df=pd.DataFrame(ord_encoded,columns=enc.get_feature_names_out(categorical_columns))
ord_test=pd.DataFrame(ord_tested,columns=enc.get_feature_names_out(categorical_columns))
X_encoded=pd.concat([X,ord_df],axis=1)
test_encoded=pd.concat([test,ord_test],axis=1)
X_encoded=X_encoded.drop(categorical_columns,axis=1)
test_encoded=test_encoded.drop(categorical_columns,axis=1)

In [ ]:
X_encoded.describe()

In [ ]:
X_encoded.info()

In [ ]:
X_encoded[numerical_columns].describe()

In [ ]:
X_encoded

In [ ]:
X_encoded.shape,test_encoded.shape

In [ ]:
from sklearn.preprocessing import MinMaxScaler

In [ ]:
scaler=MinMaxScaler()
X_encoded_scaled=scaler.fit_transform(X_encoded)
test_encoded_scaled=scaler.fit_transform(test_encoded)

In [ ]:
X_encoded_scaled.shape,X_encoded.shape,test_encoded_scaled.shape,test_encoded.shape

In [ ]:
X_encoded

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X_train,X_valid,y_train,y_valid=train_test_split(X_encoded_scaled,y,test_size=0.2,random_state=1)

In [ ]:
len(X_train)

In [ ]:
len(X_train),len(X_valid),len(y_train),len(y_valid)

In [ ]:
X_valid.shape,test_encoded.shape

In [ ]:
from sklearn.linear_model import LinearRegression
model_1=LinearRegression()
model_1.fit(X_train,y_train)
y_predict=model_1.predict(X_valid)

In [ ]:
from sklearn.metrics import mean_absolute_error as mae

In [ ]:
mae(y_predict,y_valid)

In [ ]:
y_valid

In [ ]:
y_predict

### 1.Feature Engineering 
* Polynomial Features 

In [ ]:
from sklearn.preprocessing import PolynomialFeatures

In [ ]:
poly=PolynomialFeatures(degree=2,include_bias=False)
X_train_poly=poly.fit_transform(X_train)
X_valid_poly=poly.transform(X_valid)
test_poly=poly.transform(test_encoded_scaled)

In [ ]:
model_1.fit(X_train_poly,y_train)
y_pred_1=model_1.predict(X_valid_poly)

In [ ]:
mae(y_valid,y_pred_1)

By introducing polynomial features and interaction terms, your model can better capture the complexity of the relationships in your data, leading to a significant reduction in prediction error.

In [ ]:
X_train

### Correlation Analysis

In [ ]:
X_train

### 3.Variance Threshold
* Remove features with low variance, as they provide little information to the model.

In [ ]:
from sklearn.feature_selection import VarianceThreshold

In [ ]:
selector=VarianceThreshold(threshold=0.01)
X_train_var=selector.fit_transform(X_train)
X_valid_var=selector.transform(X_valid)
test_var=selector.transform(test_encoded_scaled)

In [ ]:
model_1.fit(X_train_var,y_train)
y_pred_2=model_1.predict(X_valid_var)

In [ ]:
mae(y_valid,y_pred_2)

### 4. Feature Importance using tree-based models
* Use feature importance scores from Random Forests or XGBoost to select important features.

In [ ]:
from sklearn.ensemble import RandomForestRegressor

In [ ]:
rf_model=RandomForestRegressor(random_state=1)
rf_model.fit(X_train,y_train)

In [ ]:
#Get important features
importances=rf_model.feature_importances_

#Sort features by importance
indices=np.argsort(importances)[::-1]

# print the feature ranking
for f in range(X_train.shape[1]):
    print(f"{f+1}. Feature {indices[f]}({importances[indices[f]]})")

In [ ]:
#Select top n features 
n_top_features=30
top_features=indices[:n_top_features]

In [ ]:
model_1.fit(X_train,y_train)
y_pred_3=model_1.predict(X_valid)

In [ ]:
mae(y_valid,y_pred_3)

In [ ]:
# Create polynomial features for selected features
poly = PolynomialFeatures(degree=2, include_bias=False)
X_train_poly_rfe = poly.fit_transform(X_train)
X_valid_poly_rfe = poly.transform(X_valid)
test_poly_rfe=poly.transform(test_encoded_scaled)

# Train the linear regression model
model_1.fit(X_train_poly_rfe, y_train)

# Predict on the test set
y_pred_5 = model_1.predict(X_valid_poly_rfe)

# Calculate MAE
mae(y_valid,y_pred_5)

#### 6.Hyperparameter tunning using Grid Search and Random search

In [ ]:
from sklearn.linear_model import Ridge,Lasso
from sklearn.model_selection import GridSearchCV

In [ ]:
param_grid={'alpha':[0.1,1,10,100,1000]}

ridge=Ridge()
ridge_cv=GridSearchCV(ridge,param_grid,cv=5,scoring='neg_mean_absolute_error')
ridge_cv.fit(X_train,y_train)

In [ ]:
y_pred_ridge=ridge_cv.predict(X_valid)
mae_ridge=mae(y_valid,y_pred_ridge)
print(f"Ridge Regression MAE:{mae_ridge}")

#### 7.Ensemble Methods
* Bagging
* Boosting

In [ ]:
from sklearn.ensemble import GradientBoostingRegressor

#Train a Gradient Boosting Regressor
gbr=GradientBoostingRegressor()

gbr.fit(X_train,y_train)
y_pred_gbr=gbr.predict(X_valid)
mae_gbr=mae(y_valid,y_pred_gbr)

print(f"Gradient Boosting Regressor MAE:{mae_gbr}")

# Combine Prediction 
y_pred_combined=(y_pred_ridge+y_pred_gbr)/2
mae_combined=mae(y_valid,y_pred_combined)
print(f"Combined Model MAE:{mae_combined}")

In [ ]:
mae_gbr

In [ ]:
mae_ridge

#### 7.Advanced Models

In [ ]:
import xgboost as xgb

xgbr=xgb.XGBRegressor()
xgbr.fit(X_train,y_train)

#Predict on the valid set
y_pred_xgbr=xgbr.predict(X_valid)
mae_xgbr=mae(y_valid,y_pred_xgbr)
print(f"XGBoost Regressor MAE:{mae_xgbr}")

#### 8.Cross Validation

#### 9.Hyperparameters Tuning for XGBoost

In [ ]:
import xgboost as xgb
from sklearn.model_selection import GridSearchCV

In [ ]:
sample_submission

In [ ]:
gbr.predict(test_encoded_scaled)

In [ ]:
sample_submission= pd.read_csv(r'C:\Users\suhas\OneDrive\Documents\projects\prodigy\first prj\house-prices-advanced-regression-techniques\sample_submission.csv')
sample_submission['SalePrice']=gbr.predict(test_encoded_scaled)
sample_submission.to_csv(r'C:\Users\suhas\OneDrive\Documents\projects\prodigy\first prj',index=False)

In [ ]:
sample_submission.head()